In [2]:
!pip install pandas scikit-learn pythainlp emoji tensorflow numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=ade6dc293bd03617a61c9b6a8406ce48d17a0a2015cf53bab1ee05061de2e7df
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import unicodedata
import pythainlp
import re
import emoji
import tensorflow as tf
import numpy as np

In [4]:
def normalize_double_quote(text: str):
    all_quote = [
    '“', # U+201c
    '”', # U+201d
    "„", # U+201e
    '«', # U+00AB
    '»', # U+00BB
    '„', # U+201E
    '“', # U+201C
    '‟', # U+201F
    '”', # U+201D
    '❝', # U+275D
    '❞', # U+275E
    '〝', # U+301D
    '〞', # U+301E
    '〟', # U+301F
    '＂', # U+FF02
    ]
    std_quote = "\"" #U+0022
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def normalize_single_quote(text: str):
    all_quote = ['\u02BB', '\u02BC', '\u066C', '\u2018', '\u2019', '\u201A', '\u275B', '\u275C']
    std_quote = "\'"
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|
[^\s()]*?\)|
)+(?:
[^\s()]*?\)|
|[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def preprocess_func(text):
    new_text = unicodedata.normalize('NFKD', text)
    new_text = pythainlp.util.normalize(new_text)
    new_text = normalize_double_quote(new_text)
    new_text = normalize_single_quote(new_text)

    new_text = new_text.lower().strip()
    new_text = replace_url(new_text)
    new_text = replace_rep(new_text)

    res = [word for word in pythainlp.word_tokenize(new_text) if word and not re.search(pattern=r"\s+", string=word)]

    res = ungroup_emoji(res)
    return res

In [5]:
train_df = pd.read_csv('./train_data_fake_news.csv')
val_df = pd.read_csv('./valid_data_fake_news.csv')
test_df = pd.read_csv('./test_data_fake_news.csv')

In [6]:
text_train = [' '.join(preprocess_func(sent)) for sent in train_df['text']]
text_val = [' '.join(preprocess_func(sent)) for sent in val_df['text']]
text_test = [' '.join(preprocess_func(sent)) for sent in test_df['text']]

In [7]:
word_count = []
for sent in text_train:
    for word in sent.split():
        word_count.append(word)

In [8]:
VOCAB_SIZE = len(set(word_count))
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(text_train)

In [9]:
VOCAB_SIZE

25273

In [24]:
y_train = train_df['label'].astype(str)
y_val = val_df['label'].astype(str)
y_test = test_df['label'].astype(str)
y_class = ['0', '1']

In [11]:
y_train

0       1
1       1
2       1
3       0
4       1
       ..
2358    1
2359    1
2360    0
2361    0
2362    1
Name: label, Length: 2363, dtype: object

In [25]:
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

le = preprocessing.LabelEncoder()
le.fit(y_class)
y_train = le.transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)
print(y_train.shape)

(2363, 2)


In [13]:
train_df['label'].values.tolist()[0]

1

In [14]:
y_train[0]

array([0., 1.], dtype=float32)

In [15]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 512)         12273152  
                                                                 
 bidirectional (Bidirectiona  (None, 512)              1574912   
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 13,913,986
Trainable params: 13,913,986
No

In [17]:
sample_text = ('ร้านอาหารร้านนี้อร่อยมาก')
predictions = model.predict(np.array([sample_text]))
print(predictions)

1/1 [==============================] - 3s 3s/step
[[0.5014064  0.49859357]]


In [18]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [20]:
from tqdm.keras import TqdmCallback
model.fit(np.array(text_train), y_train, validation_data=(np.array(text_val), y_val), epochs=5, verbose=2)

Epoch 1/5
74/74 - 1753s - loss: 0.2819 - accuracy: 0.9099 - val_loss: 0.2993 - val_accuracy: 0.8874 - 1753s/epoch - 24s/step
Epoch 2/5
74/74 - 1750s - loss: 0.1803 - accuracy: 0.9424 - val_loss: 0.2787 - val_accuracy: 0.8972 - 1750s/epoch - 24s/step
Epoch 3/5
74/74 - 1790s - loss: 0.1029 - accuracy: 0.9687 - val_loss: 0.3175 - val_accuracy: 0.8893 - 1790s/epoch - 24s/step
Epoch 4/5
74/74 - 1791s - loss: 0.0579 - accuracy: 0.9865 - val_loss: 0.3183 - val_accuracy: 0.8953 - 1791s/epoch - 24s/step
Epoch 5/5
74/74 - 1757s - loss: 0.0307 - accuracy: 0.9928 - val_loss: 0.3805 - val_accuracy: 0.8992 - 1757s/epoch - 24s/step


In [21]:
model.save("rnn_lstm_model")

In [22]:
!zip -r '/content/rnn_lstm_model.zip"' '/content/rnn_lstm_model'

  adding: content/rnn_lstm_model/ (stored 0%)
  adding: content/rnn_lstm_model/saved_model.pb (deflated 88%)
  adding: content/rnn_lstm_model/assets/ (stored 0%)
  adding: content/rnn_lstm_model/variables/ (stored 0%)
  adding: content/rnn_lstm_model/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: content/rnn_lstm_model/variables/variables.index (deflated 65%)
  adding: content/rnn_lstm_model/keras_metadata.pb (deflated 89%)


In [31]:
value = model.predict(np.array(text_test))
y_test_pred = np.argmax(value,axis=1)
y_test_true = np.argmax(y_test,axis=1)
print(classification_report(y_test_true, y_test_pred))

16/16 [==============================] - 67s 4s/step
              precision    recall  f1-score   support

           0       0.86      0.85      0.86       212
           1       0.90      0.90      0.90       295

    accuracy                           0.88       507
   macro avg       0.88      0.88      0.88       507
weighted avg       0.88      0.88      0.88       507



In [32]:
loaded_model = tf.keras.models.load_model("./rnn_lstm_model")

In [33]:
value = loaded_model.predict(np.array(text_test))
y_test_pred = np.argmax(value,axis=1)
y_test_true = np.argmax(y_test,axis=1)
print(classification_report(y_test_true, y_test_pred))

16/16 [==============================] - 68s 4s/step
              precision    recall  f1-score   support

           0       0.86      0.85      0.86       212
           1       0.90      0.90      0.90       295

    accuracy                           0.88       507
   macro avg       0.88      0.88      0.88       507
weighted avg       0.88      0.88      0.88       507

